# logging模块

## 1. logger记录器


### 1.1 root logger，根记录器
2种获取方式
1. logging.getLogger()
2. logging.root

In [16]:
import logging

print(logging.getLogger() == logging.root)

root_logger = logging.getLogger()
root_logger.info("这是一条告警日志信息")

True


### 1.2 创建/获取logger

    通过getLogger()方法获取。

1. getLogger()没有任何参数的时候，返回root logger。
2. 如果传入logger名称，logger已经存在，则返回；如果不存在，则创建一个logger再返回。

```python
import logging
docs_logger = logging.getLogger("DOCS")
```

In [3]:
import logging
docs_logger = logging.getLogger("DOCS")
docs_logger.warning("这是一条Docs日志信息")

这是一条Docs日志信息


### 1.3. 常用属性和方法

**属性**
1. name：logger名称,根logger的名称就是"root"
2. level：logger的等级
3. parent: 父logger
4. handlers: 控制器列表，下面介绍
5. propagate: bool,默认值为True,True-向上传播;False-不向上传播

**方法**
1. 记录日志的方法：debug(msg, *args, **kwargs)/info()/warning()/error()/critical()
    1. msg: str, 字符串，日志的内容；
    2. *args: 和msg配合使用，使用C语言printf风格来格式化msg,用得较少
    3. **kwargs: exc_info记录堆栈信息
2. 记录日志的通用方法：log(level, msg, *args, **kwargs)
3. 设置日志等级: setLevel(),参数可以是logging.DEBUG/logging.INFO/logging.WARNING/logging.ERROR/logging.CRITICAL
4. 添加控制器：addHandler()
5. 删除控制器：removeHandler()
6. 清空控制器: logger.handlers = []

In [2]:
def print_logger_info(logger):
    print("-"*80)
    print(f"Logger name: {logger.name}, level: {logging.getLevelName(logger.level)}, \
    parent: {logger.parent}")
    for handler in logger.handlers:
        print(f"  Handler: {handler}")
        for key, value in handler.__dict__.items():
            print(f"      {key}: {value}")
    if not logger.handlers:
        print('handlers is empty!')
    print("-"*80)
print_logger_info(root_logger)

NameError: name 'root_logger' is not defined

In [12]:
print_logger_info(docs_logger)

--------------------------------------------------------------------------------
Logger name: DOCS, level: INFO,     parent: <RootLogger root (WARNING)>
  Handler: <StreamHandler stderr (INFO)>
      filters: []
      _name: None
      level: 20
      formatter: None
      _closed: False
      lock: <unlocked _thread.RLock object owner=0 count=0 at 0x108183440>
      stream: <ipykernel.iostream.OutStream object at 0x107a94b20>
--------------------------------------------------------------------------------


### 1.4 父子关系

所有logger的祖先是root logger。

1. 继承配置：子记录器可以继承父记录器的配置，包括处理器（handlers）、格式化器（formatters）和级别（levels）。
2. 日志传播：子记录器的日志消息可以传播到父记录器，从而实现集中化的日志处理,通过logger的 **propagate** 确定是否传播。
3. 分层管理：通过层次结构，可以更好地管理和组织不同模块或组件的日志记录。


```shell
ROOT
    A
    A.B
    A.B.C
    A.B.D
    A.BB
```

## 2. handler控制器

作用是控制日志打印到哪里。常用的有StreamHandler和FileHandler。

### 2.1 常用属性和方法

**属性**

1. level: 日志等级
2. formatter: 格式器
3. filters[]: 过滤器

**方法**

1. setLevel(level): 设置日志等级
2. setFormatter(fmt): 设置格式器
3. addFilter(): 添加过滤器
4. removeFilter(): 删除过滤器


### 2.2 logging.StreamHandler()

**参数**

1. stream: 默认输出在sys.stderr，也就是控制台。

```python
import logging
handler_s01 = logging.StreamHandler()
logger.addHandler(handler_s01)
```

In [17]:
import logging
docs_logger = logging.getLogger("DOCS")
docs_logger.setLevel(logging.ERROR)

handler_s01 = logging.StreamHandler()
handler_s01.setLevel(logging.INFO)

docs_logger.handlers = []
docs_logger.addHandler(handler_s01)
docs_logger.warning("这是一条Docs日志信息")

In [13]:
print_logger_info(docs_logger)

--------------------------------------------------------------------------------
Logger name: DOCS, level: INFO,     parent: <RootLogger root (WARNING)>
  Handler: <StreamHandler stderr (INFO)>
      filters: []
      _name: None
      level: 20
      formatter: None
      _closed: False
      lock: <unlocked _thread.RLock object owner=0 count=0 at 0x108183440>
      stream: <ipykernel.iostream.OutStream object at 0x107a94b20>
--------------------------------------------------------------------------------


### 2.3 logging.FileHandler()

**参数**
1. filename: str，文件路径（相对路径和绝对路径都可以）
2. mode: 模式，默认"a",[模式说明](https://docs.python.org/zh-cn/3/library/functions.html#filemodes)
    - 'r': 读取
    - 'w': 写入
    - 'x': 排它性创建，如果文件存在则失败
    - 'a': 打开文件用于写入，如果文件已经存在则在后面添加
    - 'b': 二进制模式
    - 't': 文本模式(默认)
    - '+': 打开用于更新(读取与写入)
3. encoding: str, 日志编码，默认值是None，utf-8、ascii、latin-1 ...
4. delay: bool,什么时候打开日志文件，False，日志文件在FileHandler实例化的时候打开；True：有消息需要写入的时候打开（也就是延迟打开文件），可以节省一点文件资源。
5. errors: str, 默认值是'strict',指定在编码错误时的处理方式，可以使用的值包括 'strict'、'ignore'、'replace' 等，类似于 open() 函数的 errors 参数。
    - strict：默认值，遇到编码错误时会引发 UnicodeEncodeError。
    - ignore：忽略编码错误。
    - replace：用替代字符（通常是 ?）替换编码错误。

In [1]:
import logging
docs_logger = logging.getLogger("DOCS")
docs_logger.setLevel(logging.INFO)

handler_s01 = logging.FileHandler(
    filename="./docs_log.log"
)
handler_s01.setLevel(logging.INFO)

docs_logger.handlers = []
docs_logger.addHandler(handler_s01)
docs_logger.warning("这是一条Docs日志信息")

### 2.4 隐式处理器

**隐式处理器:**
如果root logger没有设置handlers，logging模块会默认设置一个WARNING级别的StreamHandler()。


目的：为了在没有任何配置的情况下使用logging模块也能够有输出。



## 3. formatter格式器

### 3.1 logging.Formatter()

**参数**
1. fmt： str,日志记录的格式，默认值为'%(message)s', [属性说明](https://docs.python.org/zh-cn/3/library/logging.html#logrecord-attributes)
2. datefmt: 控制日志输出中日期/时间部分的格式,默认值：'%Y-%m-%d %H:%M:%S,uuu', [符号说明](https://docs.python.org/zh-cn/3/library/time.html#time.strftime)
3. style：'%' / '{' / '$',3个选项，应用于fmt，默认值为'%'。
   
    1. '%': printf风格字符串格式化；
    2. '{': str.format()风格字符串格式化；
    3. '$': string.Template字符串格式化。
5. validate: True,不正确/不匹配的fmt和style将引发ValueError。
6. defaults：默认值字典

In [37]:
import logging
docs_logger = logging.getLogger("DOCS")
docs_logger.setLevel(logging.INFO)

handler_s01 = logging.StreamHandler()
handler_s01.setLevel(logging.INFO)

fmter = logging.Formatter(
    fmt="$asctime - $name - $levelname - $message - $custom",
    datefmt="%Y-%m-%d %H:%M:%S,%a",
    style="$",
    validate=True,
    defaults={"custom":"dxh"}
)
handler_s01.setFormatter(fmter)

docs_logger.handlers = []
docs_logger.addHandler(handler_s01)
docs_logger.warning("这是一条Docs日志信息",extra={"custom":"567"})

2024-11-25 20:30:59,Mon - DOCS - WARNING - 这是一条Docs日志信息 - 567


## 4. Filter过滤器

- 过滤器是 Python 的 logging 模块中的一个类，用于实现更复杂的日志过滤操作。
- 过滤器可以被添加到日志记录器（logger）或处理器（Handler）上。
- **多过滤器(filter)**
    1. 一个记录器(logger)或者处理器(handler)可以添加多个过滤器；
    2. 多过滤器同时都返回True时，才会记录下来。


### 4.1 logging.Filter(name='')

- name：是记录器的名称，只允许等于或者低于日志记录器层级的日志通过。
```shell
ROOT
    A
    A.B
    A.B.C
    A.B.D
    A.BB
```

In [4]:
import logging

logger_a = logging.getLogger("A")
logger_b = logging.getLogger("A.B")
logger_c = logging.getLogger("A.B.C")
logger_d = logging.getLogger("A.B.D")
logger_e = logging.getLogger("A.BB")

logger_a.setLevel(logging.DEBUG)
logger_b.setLevel(logging.DEBUG)
logger_c.setLevel(logging.DEBUG)
logger_d.setLevel(logging.DEBUG)
logger_e.setLevel(logging.DEBUG)

handler = logging.StreamHandler()
handler.setLevel(logging.DEBUG)
filter_ab = logging.Filter(name="A.B")
handler.filters=[]
handler.addFilter(filter_ab)

logger_a.handlers = []
logger_b.handlers = []
logger_c.handlers = []
logger_d.handlers = []
logger_e.handlers = []
logger_a.addHandler(handler)
logger_b.addHandler(handler)
logger_c.addHandler(handler)
logger_d.addHandler(handler)
logger_e.addHandler(handler)

logger_b.propagate = False
logger_c.propagate = False
logger_d.propagate = False
# root
logger_a.debug("This is a debug message from A")
logger_b.debug("This is a debug message from A.B")
logger_c.debug("This is a debug message from A.B.C")
logger_d.debug("This is a debug message from A.B.D")
logger_e.debug("This is a debug message from A.BB")

This is a debug message from A.B
This is a debug message from A.B.C
This is a debug message from A.B.D


### 4.2 自定义过滤器

```python
# 创建一个过滤器类，继承自 logging.Filter
class AccessFilter(logging.Filter):
    def filter(self, record):
        # 只允许INFO和WARNING级别的日志
        return record.levelname == 'INFO' or record.levelname == 'WARNING'

```

Record:
- name：记录器的名称（通常是模块的名称）。
- levelname：日志记录的级别（例如 logging.DEBUG, logging.INFO, logging.WARNING, logging.ERROR, logging.CRITICAL）。
- pathname：记录日志的源文件的完整路径。
- lineno：记录日志的源文件中的行号。
- msg：日志消息的格式字符串。

In [12]:
import logging

filter_logger = logging.getLogger("filter_logger")
filter_logger.setLevel(logging.INFO)

class AccessFilter(logging.Filter):
    def filter(self, record):
        # 只允许INFO和WARNING级别的日志
        return record.levelname == 'INFO' or record.levelname == 'WARNING'
        
access_handler = logging.FileHandler(
    filename="./access.log",
    mode="w"
)
access_handler.setLevel(logging.INFO)
access_handler.filters = []
access_handler.addFilter(AccessFilter())

error_handler = logging.FileHandler(
    filename="./error.log",
    mode="w"
)
error_handler.setLevel(logging.ERROR)

filter_logger.handlers = []
filter_logger.addHandler(access_handler)
filter_logger.addHandler(error_handler)

filter_logger.info("这是一条INFO日志信息")
filter_logger.warning("这是一条WARNING日志信息")
filter_logger.error("这是一条ERROR日志信息")

In [17]:
import logging

filter_logger = logging.getLogger("filter_logger")
filter_logger.setLevel(logging.INFO)

class AccessFilter(logging.Filter):
    def filter(self, record):
        # 只允许INFO和WARNING级别的日志
        return record.levelname == 'INFO' or record.levelname == 'WARNING'

class CustomFilter(logging.Filter):
    def filter(self, record):
        return record.msg.startswith("dxh:")
        
access_handler = logging.FileHandler(
    filename="./access.log",
    mode="w"
)
access_handler.setLevel(logging.INFO)
access_handler.filters = []
access_handler.addFilter(AccessFilter())
access_handler.addFilter(CustomFilter())

error_handler = logging.FileHandler(
    filename="./error.log",
    mode="w"
)
error_handler.setLevel(logging.ERROR)

filter_logger.handlers = []
filter_logger.addHandler(access_handler)
filter_logger.addHandler(error_handler)

filter_logger.info("dxh:这是一条INFO日志信息")
filter_logger.warning("这是一条WARNING日志信息")
filter_logger.error("这是一条ERROR日志信息")

## 5. logging的快捷方法

### 5.1 basicConfig()

快速配置root logger的方法。

```python
logging.basicConfig(filename='myapp.log', level=logging.INFO)
```

**PS: basicConfig只会在第一次执行是生效，如果root logger已经被配置过，那么basicConfig()不会再被执行。**

**参数解析：**

1. filename: 文件名
2. filemode: 日志文件的模式,默认是'a',实际上是"a+t"。[模式说明](https://docs.python.org/zh-cn/3/library/functions.html#filemodes)
    - 'r': 读取
    - 'w': 写入
    - 'x': 排它性创建，如果文件存在则失败
    - 'a': 打开文件用于写入，如果文件已经存在则在后面添加
    - 'b': 二进制模式
    - 't': 文本模式(默认)
    - '+': 打开用于更新(读取与写入)
3. format: 日志格式，默认值为'%(levelname)s:%(name)s:%(message)s'，[属性说明](https://docs.python.org/zh-cn/3/library/logging.html#logrecord-attributes)
4. datefmt: 日期/时间格式,%Y-%m-%d %H:%M:%S, [符号说明](https://docs.python.org/zh-cn/3/library/time.html#time.strftime)
5. style: '%' / '{' / '$'
   
   - '%': printf风格字符串格式化【默认】；
   - '{': str.format()风格字符串格式化；
   - '$': string.Template字符串格式化。
     
7. level: 日志级别
8. handlers[]: 控制器列表

In [4]:
import logging

logging.basicConfig(
    # filename='dxh_app.log', 
    level=logging.INFO,
    handlers=[logging.StreamHandler()]
)

root_logger = logging.getLogger()
root_logger.info("这是一条INFO日志信息")
print_logger_info(root_logger)

INFO:root:这是一条INFO日志信息


--------------------------------------------------------------------------------
Logger name: root, level: INFO,     parent: None
  Handler: <StreamHandler stderr (NOTSET)>
      filters: []
      _name: None
      level: 0
      formatter: <logging.Formatter object at 0x10425e1e0>
      _closed: False
      lock: <unlocked _thread.RLock object owner=0 count=0 at 0x104108040>
      stream: <ipykernel.iostream.OutStream object at 0x103bb2fb0>
--------------------------------------------------------------------------------


In [3]:
def print_logger_info(logger):
    print("-"*80)
    print(f"Logger name: {logger.name}, level: {logging.getLevelName(logger.level)}, \
    parent: {logger.parent}")
    for handler in logger.handlers:
        print(f"  Handler: {handler}")
        for key, value in handler.__dict__.items():
            print(f"      {key}: {value}")
    if not logger.handlers:
        print('handlers is empty!')
    print("-"*80)

### 5.2 debug()/info()/warning()/error()/critical()

```python
# 参数解析
logging.debug(msg, *args, **kwargs)
```

1. msg: str, 字符串，日志的内容；
2. *args: 和msg配合使用，使用C语言printf风格来格式化msg,用得较少
3. **kwargs: exc_info记录堆栈信息

In [2]:
### logging.warning()发生了什么？
# 运行两次如下代码，看看输出是否相同？如果不同，解析一下原因？
import logging

root_logger = logging.getLogger()
root_logger.warning("这是一条日志报警信息")
logging.warning("这是一条日志报警信息_2")

## 6. 企业级日志配置
```python
logging.config.dictConfig(logging_config)
```

### 6.1 json格式
logging_config的schema信息结构(json格式)：

```json
{
  "version": 1,
  "disable_existing_loggers": false,
  "formatters": {
    "standard": {
      "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s"
    }
  },
  "handlers": {
    "app_handler": {
      "class": "logging.FileHandler",
      "filename": "app.log",
      "formatter": "standard",
      "level": "INFO"
    }
  },
  "loggers": {
    "": {
      "handlers": [
        "app_handler"
      ],
      "level": "INFO",
      "propagate": true
    },
    "docs": {
      "handlers": [
        "app_handler"
      ],
      "level": "INFO",
      "propagate": false
    }
  }
}

```

### 6.2 yaml格式

yaml格式相对于json格式，有如下优点：
- **更易读**：YAML 的语法设计使其更接近自然语言，减少了冗余的符号（如大括号、方括号和逗号），使得配置文件更易读。
- **更易写**：由于 YAML 的简洁性，编写和维护 YAML 文件通常比 JSON 文件更容易。

```shell
# 使用使用yaml第3方包加载
pip install pyyaml
```

schema信息结构：
```yml
disable_existing_loggers: false
formatters:
  standard:
    format: '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
handlers:
  app_handler:
    class: logging.FileHandler
    filename: ai_services.log
    formatter: standard
    level: INFO
loggers:
  '':
    handlers:
    - app_handler
    level: INFO
    propagate: true
  docs:
    handlers:
    - app_handler
    level: INFO
    propagate: false
version: 1

```